In [1]:
## imports and fixed parameters

import ipywidgets as widgets
from IPython.display import display
from dome_connector import Connector
from cmcl_jobsender import JobSender
from datetime import datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import folium
import matplotlib.pyplot as plt
import matplotlib as mpl

local = True
if local:
    # local deployment
    DOME_URL = 'https://cmcl.dome40.io/' # DOME platform URL
    API_KEY = "6f7d3ae990.fb00950be9a1439d80c6baf515e1f112" # Identify user
    CMCL_URL = 'http://192.168.1.171:4242/' # URL of CMCL server
    SHIP_CONN_UUID = "fb8490c8-a71a-42c4-bd67-6052cf347f2e" # UUID of DOME connector for ship data
else:
    # connect to platform
    DOME_URL = 'https://nextgen.dome40.io/'
    API_KEY = "3b35fa9591.bd62b7cddfe04b26916dd6eb95214094"
    CMCL_URL = 'https://theworldavatar.io/demos/ship-emission/'
    SHIP_CONN_UUID = "0a367029-0bf1-4c75-b5e2-182cd88bdf53"

pd.set_option('display.max_rows', None)

Textarea(value='', description='Text:', placeholder='Type something')

Button(button_style='success', description='Submit', icon='check', style=ButtonStyle(), tooltip='Submit')

In [ ]:
import os
from urllib.parse import parse_qs

# Get the query string from the environment variable
query_string = os.environ.get('QUERY_STRING', '')

# Parse the query string into a dictionary
params = parse_qs(query_string)
    
if 'mmsi' in params:    
    QUERY_MMSI = params['mmsi'][0]
else:
    raise Exception("MMSI parameter must be supplied in the form of ?mmsi=123456789 in the HTTP request.")

In [ ]:
# search DOME for ship location data - backend

# create connectors

ship_connector = Connector(DOME_URL, API_KEY, SHIP_CONN_UUID)

def parse_ship_data(json_ship):
    data = json_ship['data']
    dict_mmsi = {
        "date": [datetime.fromisoformat(data[0][j]["date"]) for j in range(len(data[0]))],
        "lat": [float(data[0][j]["lat"]) for j in range(len(data[0]))],
        "lon": [float(data[0][j]["lon"]) for j in range(len(data[0]))],
        "speed": [float(data[0][j]["speed"]) for j in range(len(data[0]))],
        "course": [float(data[0][j]["course"]) for j in range(len(data[0]))]
    }
    return dict_mmsi

def get_ship(search_string):
    search_string = search_string or "AIS"
    dict_ship = parse_ship_data(ship_connector.get_data(search_string))
    return pd.DataFrame(dict_ship)

def get_scope(df_ship):
    
    # Calculate MBR
    min_lat, max_lat = min(df_ship['lat']), max(df_ship['lat'])
    min_lon, max_lon = min(df_ship['lon']), max(df_ship['lon'])

    # Calculate Centroid
    centroid_lat = (min_lat + max_lat) / 2
    centroid_lon = (min_lon + max_lon) / 2
    
    return {"LAT": centroid_lat, "LON": centroid_lon,
            "dLAT": max((max_lat - min_lat) * 0.5 + 0.05, 0.05),
            "dLON": max((max_lon - min_lon) * 0.5 + 0.05, 0.05)}

def plot_ship(df_ship,scope):
    # Sample data: list of (latitude, longitude) tuples
    trajectory = list(zip(df_ship['lat'],df_ship['lon']))
    
    # Calculate MBR
    min_lon = scope['LON']-scope['dLON']
    max_lon = scope['LON']+scope['dLON']
    min_lat = scope['LAT']-scope['dLAT']
    max_lat = scope['LAT']+scope['dLAT']

    # Calculate Centroid
    centroid_lat = (min_lat + max_lat) / 2
    centroid_lon = (min_lon + max_lon) / 2
    centroid = (centroid_lat, centroid_lon)

    # MBR corners
    dlat = (max_lat - min_lat) * 0.02
    dlon = (max_lon - min_lon) * 0.02
    mbr_corners = [(min_lat, min_lon),
                   (min_lat, max_lon),
                   (max_lat, max_lon),
                   (max_lat, min_lon),
                   (min_lat, min_lon)]

    # Create a map centered around the first point
    m = folium.Map(location=centroid)
    
    # Add filled MBR to the map
    folium.Polygon(locations=mbr_corners,color="green",fill=True,fill_color="green",fill_opacity=0.3,weight=0).add_to(m)

    # Add points to the map
    for i in range(len(trajectory)):
        point = trajectory[i]
        folium.CircleMarker(location=point,radius=5,popup=folium.Popup(str(df_ship['date'][i])),
                            color='none', fill=True, fill_color='black', fill_opacity=0.7).add_to(m)
        
    colormap = plt.get_cmap('autumn')
    colors = [colormap(i / len(trajectory)) for i in range(len(trajectory))]

    # Add segments to the map with different colors
    for i in range(len(trajectory) - 1):
        folium.PolyLine(locations=[trajectory[i], trajectory[i + 1]],
                        color=mpl.colors.to_hex(colors[i]),  # Convert RGBA to hex
                        weight=2.5,opacity=1).add_to(m)
    
    # Fit bounds to ensure everything is visible
    m.fit_bounds([[min_lat-dlat, min_lon-dlon], [max_lat+dlat, max_lon+dlon]])
    
    return m

In [ ]:
# search DOME for ship data - frontend

output_dome_ship = widgets.Output(layout=widgets.Layout())

df_ship = get_ship(QUERY_MMSI)
scope = get_scope(df_ship)

with output_dome_ship:
    output_dome_ship.clear_output()
    display(plot_ship(df_ship,scope))

In [ ]:
# submit job to CMCL server

from uuid import uuid4

job_sender = JobSender(CMCL_URL)

# Create a text area widget
text_area_job_label = widgets.Text(
    placeholder=f"""MMSI:{QUERY_MMSI}""",
    description="""Name of simulation: """,
    layout=widgets.Layout(flex_flow='row',width='70%'),
    style={'description_width': 'initial'},
    disabled=False
)

slider_job_step = widgets.IntSlider(
    value=0,
    min=1,
    max=len(df_ship['date']),
    description="""Number of timesteps: """,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    layout=widgets.Layout(flex_flow='row',width='70%'),
    style={'description_width': 'initial'},
    disabled=False
)

text_area_job_password = widgets.Password(
    placeholder="""******""",
    description="""Password: """,
    layout=widgets.Layout(flex_flow='row',width='70%'),
    style={'description_width': 'initial'},
    disabled=False
)

# Create a button widget
button_job = widgets.Button(
    description='Submit',
    button_style='warning',
    tooltip='Submit',
    icon='circle'
)

# Define a function to handle the button click event
def click_button_job(b):
    
    ob_style = b.button_style
    ob_description = b.description
    
    b.button_style = 'info'  # Change the button style
    b.description = 'Clicked!'  # Change the button description
    b.icon = 'check'

    with output_job:
        output_job.clear_output()
        
        if len(text_area_job_label.value)>0:
            label = text_area_job_label.value
        else:
            label = f"MMSI:{QUERY_MMSI}"
            
        try:
            num_step = slider_job_step.value
        except:
            print('Cannot parse number of steps, assume to be 0.')
            num_step = 0
        
        if num_step == 0:
            print('Number of timestep = 0 i.e. all timesteps will be simulated.')
            num_step = len(df_ship['date'])
            
        
        df_ship['dates'] = pd.to_datetime(df_ship['date'])
        list_timestep = [x.to_pydatetime() for x in df_ship['dates']]
        
        if text_area_job_password.value == os.getenv('NB_PW', uuid4()):
            print('Sending ship data...')
            response = job_sender.add_ship_data(QUERY_MMSI,df_ship)
            if response.status_code == 200:
                print('Ship data successfully sent.')
                print(job_sender.run_simulation_without_ship(label,scope,list_timestep,num_step))
        else:
            print('No correct password is supplied.')
            print('*** MOCK SIMULATION OUTPUT ***')
            print('Sending ship data...')
            print('Ship data successfully sent.')
            print(f'Running simulation {label}...')
            count = 0
            for t in list_timestep:
                if count < num_step:
                    print(f'Simulating {label} at {t}')
                    count = count + 1
            print('Complete.')

    b.button_style = ob_style  # Change the button style
    b.description = ob_description  # Change the button description

output_job = widgets.Output()
output_job.layout.overflow = 'auto'
output_job.layout.height = '100px'

# Set the event handler for the button click event
button_job.on_click(click_button_job)

In [ ]:
# final-layout

label_dome_ship = widgets.HTML(
value=f'<p style="font-size:24px; font-weight:bold;text-align:center">Location data of ship MMSI:{QUERY_MMSI}</p>'
)

vbox_dome_ship = widgets.VBox(
    [label_dome_ship, output_dome_ship],
    layout=widgets.Layout(width="50%",align_items='center'))

########################

label_job = widgets.HTML(
value='<p style="font-size:24px; font-weight:bold;text-align:center">Start new simulation</p>'
)

vbox_job = widgets.VBox(
    [label_job, text_area_job_label, slider_job_step,
        text_area_job_password, button_job, output_job],
    layout=widgets.Layout(width="50%",align_items='center'))

table = widgets.HBox([vbox_dome_ship, vbox_job],
                     layout=widgets.Layout(align_items='flex-start'), width='100%')

display(table)